In [1]:
import numpy as np
import pandas as pd
from pylab import plot,show,hist,bar
import gc
%config IPCompleter.greedy=True


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("../data/train.csv",sep="\t")
#data = pd.read_csv("../data/test.csv",sep="\t")
print len(data)
#data = data.sample#(frac=0.5)
print len(data)

1482535
1482535


# Overall description 

### Observations
1. Few category names missing
2. 42% of brand_name in missing 1
3. category_name , brand_name , item_description have some missing values

In [4]:
data['log_price'] = np.log(data.price+1)


### observation
1. Price is highly skewed
2. Log price looks good

# Item_condition_id

In [5]:
#data['median_item_condition_id']=data.item_condition_id.map({1:18.0,2:17.0,3:16.0,4:15.0,5:19.0})
#print data.item_condition_id

it_id_rv=data[['log_price','item_condition_id']].groupby(['item_condition_id']).describe()
it_id_rv_dict = dict(zip(it_id_rv['log_price']['mean'].index,it_id_rv['log_price']['mean']))
data['mean_it_id']=map(lambda x:it_id_rv_dict[x],data.item_condition_id)
del it_id_rv
del it_id_rv_dict

### observation 
1. 5 condition is really low , but has the highest median price ( maybe means good condition )
2. 4 count is low , even its median price is low ( maybe means really bad condition )

###  observation 
1. from log price plot we see , that almost for all categories the distribution is same except that it is somewhat different for 5th

# Category_name  





In [6]:
data['Cat1']=data.category_name.str.split('/').str.get(0)
data['Cat2']=data.category_name.str.split('/').str.get(1)
data['Cat3']=data.category_name.str.split('/').str.get(2)
data.Cat1= data.Cat1.fillna('NotPresent')
data.Cat2= data.Cat2.fillna('NotPresent')
data.Cat3= data.Cat3.fillna('NotPresent')

In [7]:
cat1_rv=data[['log_price','Cat1']].groupby(['Cat1']).describe()
cat1_rv_dict = dict(zip(cat1_rv['log_price']['mean'].index,cat1_rv['log_price']['mean']))
data['mean_Cat1']=map(lambda x:cat1_rv_dict[x],data.Cat1)
#print data.mean_Cat1
del cat1_rv
del cat1_rv_dict

In [8]:
cat2_rv=data[['log_price','Cat2']].groupby(['Cat2']).describe()
cat2_rv_dict = dict(zip(cat2_rv['log_price']['mean'].index,cat2_rv['log_price']['mean']))
data['mean_Cat2']=map(lambda x:cat2_rv_dict[x],data.Cat2)
del cat2_rv
del cat2_rv_dict

In [9]:
cat3_rv=data[['log_price','Cat3']].groupby(['Cat3']).describe()
cat3_rv_dict = dict(zip(cat3_rv['log_price']['mean'].index,cat3_rv['log_price']['mean']))
data['mean_Cat3']=map(lambda x:cat3_rv_dict[x],data.Cat3)
del cat3_rv
del cat3_rv_dict

In [10]:
data=data.drop(['Cat1','Cat2','Cat3','category_name'],axis=1)
gc.collect()

273

### Cat1

len(data.Cat1.unique())

plot(xrange(10),data[['price','Cat1']].groupby(['Cat1']).median(),'-x')
show()

data[['price','Cat1']].groupby(['Cat1']).describe()#.sort_values(by='count', ascending=False)

hist(np.log(data[data['Cat1']=='Men'].price+1))
show()

.

##  shipping


In [11]:
sp_rv=data[['log_price','shipping']].groupby(['shipping']).describe()
sp_rv_dict = dict(zip(sp_rv['log_price']['mean'].index,sp_rv['log_price']['mean']))
data['mean_shipping']=map(lambda x:sp_rv_dict[x],data.shipping)
del sp_rv
del sp_rv_dict

##  name

In [12]:
data['len_name']=list(map(lambda x : len(x),data.name))

## brand_name

In [13]:
data.brand_name= data.brand_name.fillna('NotPresent')

In [14]:
brand_name_rv=data[['brand_name','log_price']].groupby(['brand_name']).describe()
brand_name_rv_dict = dict(zip(brand_name_rv['log_price']['mean'].index,brand_name_rv['log_price']['mean']))
data['mean_brand_name']=map(lambda x:brand_name_rv_dict[x],data.brand_name)
del brand_name_rv
del brand_name_rv_dict

##  item_description 


In [15]:
data.item_description=data.item_description.fillna('No description yet')

In [16]:
len_desc = list(map(lambda x:len(x),data.item_description))


In [ ]:
i=12
print data[data.price>1500].price.iloc[i]
print data[data.price>1500].brand_name.iloc[i]
print data[data.price>1500].item_description.iloc[i]
print data[data.price>1500].name.iloc[i]

# Pairwise


In [ ]:
aa = data[['log_price','brand_name','Cat1','Cat2','Cat3','item_condition_id']].groupby(['brand_name','item_condition_id','Cat1','Cat2','Cat3']).mean().sort_values(['log_price'],ascending=False)['log_price'].to_dict()

In [ ]:
data['letssee']=list(map(lambda x:aa[(x.brand_name,x.Cat1,x.Cat2,x.Cat3,x.item_condition_id)],data))

In [ ]:
map(lambda y:y[1][1],data.iloc[:10].iterrows())

In [ ]:
data.iteritems()[0]


##  Prediction 

data.Cat1= data.Cat1.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat1)
print len(le.classes_)
data['Cat1Tf'] = le.transform(data.Cat1) 


data.Cat2= data.Cat2.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat2)
print len(le.classes_)
data['Cat2Tf'] = le.transform(data.Cat2) 


data.Cat3= data.Cat3.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat3)
print len(le.classes_)
data['Cat3Tf'] = le.transform(data.Cat3) 


data.brand_name=data.brand_name.fillna('OTHERS')
le=LabelEncoder()
le.fit(data.brand_name)
print len(le.classes_)
data['brand_nameTf']= le.transform(data.brand_name) 


data['log_price'] = np.log(data.price+1)


In [17]:
data1 = data.drop(['item_condition_id','price','train_id','name','brand_name','item_description'],axis=1)
print data1.columns
#train, test = train_test_split(data, test_size=0.7)
#print train.columns
del data
gc.collect()

Index([u'shipping', u'log_price', u'mean_it_id', u'mean_Cat1', u'mean_Cat2',
       u'mean_Cat3', u'mean_shipping', u'len_name', u'mean_brand_name'],
      dtype='object')


151

In [ ]:
len(data[data.Cat1.isnull()==True])

In [18]:
trn_columns = [u'shipping', u'mean_it_id', u'mean_Cat1', u'mean_Cat2',
       u'mean_Cat3', u'mean_shipping', u'len_name', u'mean_brand_name']
targ = ['log_price']

In [19]:
train,test = train_test_split(data1,test_size=0.3)

In [20]:
del data1
gc.collect()

20

### LinearRegression

In [21]:
lr=LinearRegression()
lr.fit(train[trn_columns],train[targ])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [22]:
preds = lr.predict(test[trn_columns])
print len(preds)

444761


In [23]:
np.sqrt(np.mean(np.square(preds-test[targ])))

log_price    0.595722
dtype: float64

### Ridge

In [24]:
from sklearn.linear_model import Ridge
rg=Ridge()
rg.fit(train[trn_columns],train[targ])

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [25]:
preds = rg.predict(test[trn_columns])
print len(preds)

444761


In [26]:
np.sqrt(np.mean(np.square(preds-test[targ])))

log_price    0.595721
dtype: float64

### SVR

In [ ]:
svr = LinearSVR()
svr.fit(train[trn_columns],train[targ])

In [ ]:
preds = svr.predict(test[trn_columns])
print len(preds)

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

In [ ]:
svrr = SVR()
svrr.fit(train[trn_columns],train[targ])

In [ ]:
preds = svrr.predict(test[trn_columns])
print len(preds)

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

### RandomForest

In [27]:
gbr=GradientBoostingRegressor()
gbr.fit(train[trn_columns],train[targ])

C:\Python27\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [28]:
preds = gbr.predict(test[trn_columns])

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))


0.576081285323


In [ ]:
import gc
gc.collect()
print np.sqrt(np.mean(np.square(preds-test[targ])))

### Adaboost

In [ ]:
abr=AdaBoostRegressor()
abr.fit(train[trn_columns],train[targ])

In [ ]:
preds = abr.predict(test[trn_columns])

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

###  SGD

In [ ]:
sgd = SGDRegressor( n_iter=100)
sgd.fit(train[trn_columns],train[targ])

In [ ]:
preds = sgd.predict(test[trn_columns])
print preds

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

### MLP

In [ ]:
mlp = MLPRegressor()
mlp.fit(train[trn_columns],train[targ])

In [ ]:
preds = mlp.predict(test[trn_columns])

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))